In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException
import os
import time

# Setup driver
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:5005")
chromedriver_path = '/usr/local/bin/chromedriver'  # Adjust this path as necessary
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(options=chrome_options)

# Function to take a screenshot of an element
def capture_element_screenshot(driver, url, element_selector, output_filename):
    
    time.sleep(5)  # Wait for any dynamic content to load

    try:
        # Handle any alerts that might appear
        alert = driver.switch_to.alert
        alert.dismiss()  # or alert.accept() if you need to accept the alert
        print("Alert dismissed")
    except NoAlertPresentException:
        print("No alert present")

    element = driver.find_element(By.CSS_SELECTOR, element_selector)
    if element:
        element.screenshot(output_filename)
        print(f'Screenshot saved as {output_filename}')
    else:
        print('Element not found')

# Define URL and element selector
url = 'https://member.bilibili.com/platform/upload/video/frame'
element_selector = '.geetest_holder'
output_dir = "bilibili_test"
os.makedirs(output_dir, exist_ok=True)
output_filename = os.path.join(output_dir, 'element_screenshot.png')
driver.get(url)

In [3]:
# Capture screenshot
capture_element_screenshot(driver, url, element_selector, output_filename)

# Cleanup
# driver.quit()

No alert present
Screenshot saved as bilibili_test/element_screenshot.png


In [4]:
!ls

__pycache__                  pub_xhs.py
app.py                       pub_y2b.ipynb
archived                     pub_y2b.py
automator.txt                pub_y2b_v2.ipynb
autopub.py                   run_autopub.sh
bilibili_captha.ipynb        send_email_qreader.py
bilibili_captha_old.ipynb    setup_autopub.sh
bilibili_test                shipinhao_publisher.ipynb
cropped_qr_code.jpg          shipinhao_publisher_v2.ipynb
login_douyin.py              solve_captcha_2captcha.py
login_shipinhao.py           solve_captcha_turing.py
login_xiaohongshu.py         temp
logs                         test.png
logs-autopub                 test_json.py
process_video.py             test_xdo.py
processed.csv                transcription_data
pub_bilibili.py              utils.py
pub_douyin.py                videos
pub_shipinhao.ipynb          videos_db.csv
pub_shipinhao.py             youtube_publisher.ipynb


In [5]:
!pwd

/Volumes/Lachlan/Projects/auto-publish


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoAlertPresentException
from PIL import Image
import os
import time

# Setup driver
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:5005")
chromedriver_path = '/usr/local/bin/chromedriver'  # Adjust this path as necessary
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(options=chrome_options)

# Function to take a screenshot of an element and crop it dynamically
def capture_and_crop_screenshot(driver, url, element_selector, output_filename, crop_bottom_percentage):
    # driver.get(url)
    time.sleep(5)  # Wait for any dynamic content to load

    try:
        alert = driver.switch_to.alert
        alert.dismiss()  # Dismiss any alerts that might be open
        print("Alert dismissed")
    except NoAlertPresentException:
        print("No alert present")

    element = driver.find_element(By.CSS_SELECTOR, element_selector)
    if element:
        temp_filename = 'temp_screenshot.png'
        element.screenshot(temp_filename)
        # Open the temporary screenshot and crop it
        img = Image.open(temp_filename)
        width, height = img.size
        # Calculate crop height dynamically based on the percentage of the bottom to be removed
        crop_height = int(height * (crop_bottom_percentage / 100))
        # Define the area to keep
        img_cropped = img.crop((0, 0, width, height - crop_height))
        img_cropped.save(output_filename)
        os.remove(temp_filename)  # Clean up the temporary file
        print(f'Cropped screenshot saved as {output_filename}')
    else:
        print('Element not found')

# # Define URL and element selector
# url = 'https://member.bilibili.com/platform/upload/video/frame'
# element_selector = '.geetest_holder'
# output_dir = "bilibili_test"
# os.makedirs(output_dir, exist_ok=True)
# output_filename = os.path.join(output_dir, 'cropped_element_screenshot.png')

# Capture and crop screenshot
capture_and_crop_screenshot(driver, url, element_selector, output_filename, crop_bottom_percentage=20)  # Adjust 'crop_bottom_percentage' as needed

# Cleanup
# driver.quit()


No alert present
Cropped screenshot saved as bilibili_test/element_screenshot.png


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoAlertPresentException
from PIL import Image
import os
import time

# # Setup driver
# chrome_options = Options()
# chrome_options.add_experimental_option("debuggerAddress", "localhost:5005")
# chromedriver_path = '/usr/local/bin/chromedriver'  # Adjust this path as necessary
# service = Service(executable_path=chromedriver_path)
# driver = webdriver.Chrome(options=chrome_options)

# Function to take a screenshot of an element excluding a specific inner part
def capture_and_crop_screenshot(driver, outer_element_selector, inner_element_selector, output_filename):
    try:
        alert = driver.switch_to.alert
        alert.dismiss()  # Dismiss any alerts that might be open
        print("Alert dismissed")
    except NoAlertPresentException:
        print("No alert present")

    outer_element = driver.find_element(By.CSS_SELECTOR, outer_element_selector)
    inner_element = driver.find_element(By.CSS_SELECTOR, inner_element_selector)
    if outer_element and inner_element:
        # Take screenshot of the whole element
        temp_filename = 'temp_screenshot.png'
        outer_element.screenshot(temp_filename)
        # Open the temporary screenshot and crop it
        img = Image.open(temp_filename)
        outer_element_rect = outer_element.rect
        inner_element_rect = inner_element.rect
        
        # Calculate the height to crop based on the relative top of the inner element
        new_height =  img.height - (outer_element_rect['y'] - inner_element_rect['y'])
        print(inner_element_rect['y'], outer_element_rect['y'])
        
        if new_height > 0:
            img_cropped = img.crop((0, 0, img.width, new_height))
            img_cropped.save(output_filename)
            print(f'Screenshot saved as {output_filename}')
        else:
            print("No valid crop height calculated; screenshot not cropped.")
        
        os.remove(temp_filename)  # Clean up the temporary file
    else:
        print('Element not found')

# Define element selectors and output file
outer_element_selector = '.geetest_holder.geetest_silver'
inner_element_selector = '.geetest_panel'
output_dir = "geetest_screenshot"
os.makedirs(output_dir, exist_ok=True)
output_filename = os.path.join(output_dir, 'element_screenshot.png')

# Capture and crop screenshot
capture_and_crop_screenshot(driver, outer_element_selector, inner_element_selector, output_filename)

# Cleanup
# driver.quit()


No alert present
1260.5 1348
Screenshot saved as geetest_screenshot/element_screenshot.png


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

def setup_driver():
    # Configure the driver (Assuming Chrome)
    chrome_options = Options()
    # If you are using a remote debugger, configure it here
    chrome_driver_path = '/usr/local/bin/chromedriver'  # Update to your Chromedriver path
    driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)
    return driver

def get_element_position_difference(driver, outer_element_selector, img_selector):
    # Navigate to the page (Assuming you are already there, this line is just for completeness)
    # driver.get("https://example.com")

    # Find elements
    outer_element = driver.find_element(By.CSS_SELECTOR, outer_element_selector)
    img_element = driver.find_element(By.CSS_SELECTOR, img_selector)

    # Get the bounding rectangle and calculate differences
    outer_rect = driver.execute_script("return arguments[0].getBoundingClientRect();", outer_element)
    img_rect = driver.execute_script("return arguments[0].getBoundingClientRect();", img_element)

    # Calculate vertical (top) difference
    print(img_rect['top'], outer_rect['top'])
    vertical_difference = img_rect['top'] - outer_rect['top']

    print(f"Vertical difference between outer element and image: {vertical_difference}")

    return vertical_difference

# Example usage
if __name__ == "__main__":
    # driver = setup_driver()
    outer_element_selector = '.geetest_holder.geetest_silver'  # Example, replace with your actual selector
    img_selector = '.geetest_item_img'  # Selector for the image

    # Call function to get the position difference
    diff = get_element_position_difference(driver, outer_element_selector, img_selector)
    print("Difference in Y position: ", diff)

    # Cleanup, close the browser
    # driver.quit()


137.734375 87.5
Vertical difference between outer element and image: 50.234375
Difference in Y position:  50.234375


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    chromedriver_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)
    print("Driver set up successfully.")
    return driver

def search_and_select_topic(driver, topic_name):
    print(f"Attempting to open the search dialog to search for '{topic_name}'.")

    # Use JavaScript to click on the "搜索更多话题" to open the search dialog
    search_dialog_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[contains(text(), '搜索更多话题')]"))
    )
    driver.execute_script("arguments[0].click();", search_dialog_button)
    print("Search dialog opened via JavaScript click.")
    
    time.sleep(2)

    # Enter the topic name in the search input
    search_input = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "input.bcc-search-input"))
    )
    search_input.clear()
    print("Input cleared.")
    search_input.send_keys(topic_name)
    print(f"Entered '{topic_name}' in the search box.")
    search_input.send_keys(Keys.ENTER)
    print("Search executed.")
    
    time.sleep(2)

    # Wait for the search results to update and select the topic
    topic_xpath = f"//div[contains(@class, 'topic-tag-name') and contains(text(), '{topic_name}')]"
    topic_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, topic_xpath))
    )
    driver.execute_script("arguments[0].click();", topic_element)
    print(f"Topic '{topic_name}' selected via JavaScript click.")
    
    time.sleep(2)

    # Click the "确定" button to confirm the selection using JavaScript
    confirm_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//button[.//span[contains(text(), '确定')]]"))
    )
    driver.execute_script("arguments[0].click();", confirm_button)
    print("Confirmation button clicked via JavaScript and topic added.")
    
    time.sleep(2)

# Example usage
if __name__ == "__main__":
    # driver = setup_driver()
    # driver.get("https://example.com")  # Replace with your actual page URL

    try:
        search_and_select_topic(driver, "随手记录我的生活碎片")
    finally:
        time.sleep(5)  # Allow time to observe the actions
        # driver.quit()
        print("Driver closed.")


Attempting to open the search dialog to search for '随手记录我的生活碎片'.
Search dialog opened via JavaScript click.
Input cleared.
Entered '随手记录我的生活碎片' in the search box.
Search executed.
Topic '随手记录我的生活碎片' selected via JavaScript click.
Confirmation button clicked via JavaScript and topic added.
Driver closed.
